In [7]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import os
from os.path import join
from qonnx.core.modelwrapper import ModelWrapper
    
build_dir = os.environ["FINN_BUILD_DIR"]
finn_root = os.environ["FINN_ROOT"]
MODEL_NAME = 'yololit320'
print(build_dir)
print(finn_root)

/home/vision/danilowi/serious_mot/build_dirs/yololit
/home/vision/danilowi/serious_mot/finn


In [4]:
import torch
import numpy as np

from qonnx.core.datatype import DataType
from qonnx.util.cleanup import cleanup_model
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
)
import onnx
from brevitas.export import export_qonnx
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from finn.util.pytorch import ToTensor

from models.yolo import get_model
from models.finn_models import QuantDetect

net, _ = get_model('yololit.yaml', 'best.pt', backbone_only=False)
model = net.eval()
for m in model.modules():
    if isinstance(m, QuantDetect):
        m.finn_export = True

# test input
test_input_np = (np.random.rand(1, 3, 320, 320) * 255).astype(np.uint8)
np.save(join(finn_root, "notebooks", "experiments", "yololit", "test_input_320x320.npy"), test_input_np)
# test_input_np = np.load(join(finn_root, "notebooks", "experiments", "yololit", "test_input_320x320.npy"))
test_input = torch.from_numpy(test_input_np).float() / 255.0

print('INPUT:', test_input.shape)
print(test_input)
output_golden = model(test_input)
print('OUTPUT:', [x.shape for x in output_golden] if isinstance(output_golden, list) else output_golden.shape)
print(output_golden)
for i, out in enumerate(output_golden):
    np.save(join(finn_root, "notebooks", "experiments", "yolov8", MODEL_NAME + "_output_golden_{}.npy".format(i)), out.detach())

# onnx export
model_file = join(finn_root, "notebooks", "experiments", "yololit", MODEL_NAME + ".onnx")
onnx_model = export_qonnx(model, test_input, model_file)
model = ModelWrapper(model_file)
model = cleanup_model(model)
model = model.transform(ConvertQONNXtoFINN())

# add preprocessing
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
chkpt_preproc_name = join(build_dir, "preproc.onnx")
export_qonnx(ToTensor(), torch.randn(ishape), chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = cleanup_model(pre_model)
pre_model = pre_model.transform(ConvertQONNXtoFINN())
model = model.transform(MergeONNXModels(pre_model))
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

# final cleanup
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(model_file)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:350.)
  return F.conv2d(input, weight, bias, self.stride,


INIT STRIDE = tensor([ 16.,  32.,  64., 128., 256.])
Transferred 452/452 items from best.pt
INPUT: torch.Size([1, 3, 320, 320])
tensor([[[[0.2353, 0.1373, 0.1333,  ..., 0.8000, 0.9725, 0.4353],
          [0.4078, 0.8235, 0.4235,  ..., 0.1569, 0.4902, 0.4235],
          [0.3137, 0.6314, 0.2824,  ..., 0.8863, 0.7373, 0.0824],
          ...,
          [0.8784, 0.1686, 0.9765,  ..., 0.5451, 0.1647, 0.4235],
          [0.5647, 0.1725, 0.9255,  ..., 0.6941, 0.0824, 0.4549],
          [0.9059, 0.3373, 0.5333,  ..., 0.4745, 0.2549, 0.9098]],

         [[0.2078, 0.4196, 0.6471,  ..., 0.3804, 0.1765, 0.8627],
          [0.8863, 0.5176, 0.9333,  ..., 0.4980, 0.5373, 0.7490],
          [0.1961, 0.1608, 0.1882,  ..., 0.7765, 0.8431, 0.5647],
          ...,
          [0.5137, 0.4667, 0.8471,  ..., 0.3647, 0.6039, 0.1451],
          [0.0353, 0.7804, 0.4627,  ..., 0.6000, 0.2196, 0.7333],
          [0.0745, 0.4431, 0.7137,  ..., 0.6039, 0.6784, 0.8941]],

         [[0.8706, 0.2000, 0.5333,  ..., 0.051

/home/vision/danilowi/serious_mot/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [8]:
showInNetron(model_file)

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/finn/notebooks/experiments/yololit/yololit320.onnx' at http://0.0.0.0:2222


In [31]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.general import (
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    ApplyConfig,
)
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb
import finn.transformation.streamline.reorder as reorder

model = ModelWrapper(model_file)
# model = ModelWrapper(join(finn_root, "notebooks", "experiments", "yololit", "test2.onnx"))
model = model.transform(Streamline())
model.save(join(finn_root, "notebooks", "experiments", "yololit", "test.onnx"))
model = model.transform(reorder.MoveLinearPastFork())
model = model.transform(reorder.MoveMulPastJoinAdd())
model = model.transform(reorder.MoveLinearPastFork())
model = model.transform(reorder.MoveMulPastJoinAdd())

model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model.save(join(finn_root, "notebooks", "experiments", "yololit", "test3.onnx"))

model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())

model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastJoinAdd())

model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastJoinAdd())

model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastJoinAdd())

model = model.transform(absorb.AbsorbConsecutiveTransposes())
model.save(join(finn_root, "notebooks", "experiments", "yololit", "test4.onnx"))


# def clean(model):
#     model = model.transform(GiveUniqueNodeNames())
#     model = model.transform(GiveReadableTensorNames())
#     model = model.transform(InferDataTypes())
#     model = model.transform(InferDataLayouts())

#     return model

In [32]:
showInNetron(model_file)

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/finn/notebooks/experiments/yololit/yololit320.onnx' at http://0.0.0.0:2222


In [27]:
showInNetron(join(finn_root, "notebooks", "experiments", "yololit", "test3.onnx"))

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/finn/notebooks/experiments/yololit/test3.onnx' at http://0.0.0.0:2222


In [33]:
showInNetron(join(finn_root, "notebooks", "experiments", "yololit", "test4.onnx"))

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/finn/notebooks/experiments/yololit/test4.onnx' at http://0.0.0.0:2222


In [39]:
from qonnx.transformation.infer_shapes import InferShapes
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw


model = ModelWrapper(join(finn_root, "notebooks", "experiments", "yololit", "test4.onnx"))

model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
model = model.transform(to_hw.InferVectorVectorActivation())
model = model.transform(to_hw.InferPool())
model = model.transform(to_hw.InferConvInpGen())
model = model.transform(to_hw.InferAddStreamsLayer())
model = model.transform(to_hw.InferConcatLayer())
model = model.transform(to_hw.InferSplitLayer())
model = model.transform(to_hw.InferUpsample())
model = model.transform(to_hw.InferDuplicateStreamsLayer()) 

model = model.transform(InferShapes())
model = model.transform(InferDataTypes())
model = model.transform(InferDataLayouts())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())

model.save(join(finn_root, "notebooks", "experiments", "yololit", "hw.onnx"))

In [40]:
showInNetron(join(finn_root, "notebooks", "experiments", "yololit", "hw.onnx"))

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/finn/notebooks/experiments/yololit/hw.onnx' at http://0.0.0.0:2222


In [6]:
showInNetron(join(build_dir, "yololit_fifosizing", "intermediate_models", "step_hw_ipgen.onnx"))

Stopping http://0.0.0.0:2222
Serving '/home/vision/danilowi/serious_mot/build_dirs/yololit/yololit_fifosizing/intermediate_models/step_hw_ipgen.onnx' at http://0.0.0.0:2222


In [5]:
intermediate_models = join(build_dir, "yololit_fifosizing", "intermediate_models")
!ls -tr {intermediate_models}

supported_op_partitions		      step_apply_folding_config.onnx
dataflow_parent.onnx		      step_minimize_bit_width.onnx
step_create_dataflow_partition.onnx   step_generate_estimate_reports.onnx
step_specialize_layers.onnx	      step_hw_codegen.onnx
step_target_fps_parallelization.onnx  step_hw_ipgen.onnx
